In [61]:
import json
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

data_folder = "data"

names = [x for x in os.listdir(data_folder) if not x.endswith("zip")]
discrepancy_data  = []
for name in names:
    
    path = os.path.join(data_folder, name)
    functions = [x for x in os.listdir(path) if x.endswith("json")]
    for filename in functions:
        filepath = os.path.join(path, filename)
        
        with open(filepath) as f:
            data = json.load(f)
        
        for scen in data['scenarios']:
            *x, cache = data['algorithm']['name'].split("-")
            for run_id, run in enumerate(scen['runs'], 1):
                discrepancy_data.append((
                    data['function_id'], 
                    data['algorithm']['name'],
                    scen['dimension'],
                    run_id, 
                    run['instance'],
                    run['cache_discrepancy'],
                    run['best']['evals'],
                    run['best']['y'],
                    int(cache) if cache.isdigit() else 0
                ))

data = pd.DataFrame(discrepancy_data, 
                    columns=["fid", "alg", "dimension",  
                             "run", "instance", "discrepancy", "evals", "y", "cache_size"])

In [62]:
data.query("discrepancy > 0 and alg.str.endswith('16')").groupby([ "fid", "dimension", "alg"])['discrepancy'].mean().unstack()

alg            CMA-ES-GAUSSIAN-cache-16  CMA-ES-GAUSSIAN-mirror-cache-16  \
fid dimension                                                              
1   2                          0.082297                         0.072810   
    5                          0.038380                         0.037492   
    10                         0.007219                         0.007372   
    20                         0.000204                         0.000220   
2   2                          0.085647                         0.076509   
...                                 ...                              ...   
23  20                         0.000159                         0.000163   
24  2                          0.095480                         0.068196   
    5                          0.040811                         0.037730   
    10                         0.007248                         0.007033   
    20                         0.000180                         0.000194   

alg            CMA-ES-GAUSSIAN-mirror-orthogonal-cache-16  \
fid dimension                                               
1   2                                            0.071169   
    5                                            0.033132   
    10                                           0.006985   
    20                                           0.000169   
2   2                                            0.073881   
...                                                   ...   
23  20                                           0.000188   
24  2                                            0.073873   
    5                                            0.033016   
    10                                           0.006904   
    20                                           0.000195   

alg            CMA-ES-GAUSSIAN-orthogonal-cache-16  CMA-ES-HALTON-cache-16  \
fid dimension                                                                
1   2                                     0.128551                0.040733   
    5                                     0.048542                0.031217   
    10                                    0.008257                0.006921   
    20                                    0.000227                0.000169   
2   2                                     0.132154                0.040077   
...                                            ...                     ...   
23  20                                    0.000248                0.000208   
24  2                                     0.135084                0.041112   
    5                                     0.049952                0.031405   
    10                                    0.008642                0.006593   
    20                                    0.000248                0.000158   

alg            CMA-ES-HALTON-mirror-cache-16  \
fid dimension                                  
1   2                               0.048507   
    5                               0.034598   
    10                              0.007339   
    20                              0.000168   
2   2                               0.048260   
...                                      ...   
23  20                              0.000191   
24  2                               0.046892   
    5                               0.034302   
    10                              0.006623   
    20                              0.000158   

alg            CMA-ES-HALTON-mirror-orthogonal-cache-16  \
fid dimension                                             
1   2                                          0.068948   
    5                                          0.031740   
    10                                         0.007137   
    20                                         0.000149   
2   2                                          0.070015   
...                                                 ...   
23  20                                         0.000183   
24  2                                         

In [ ]:
view = data.query(
    "discrepancy > 0 and not alg.str.contains('mirror') and not alg.str.contains('orthogonal')"
).groupby(["dimension", "alg"])['discrepancy'].mean().unstack().T.sort_index()

sns.heatmap(view, annot=True, norm=LogNorm())